In [1]:
from pandas_summary import DataFrameSummary
import redcap
import pandas as pd
import numpy as np
import macos_keychain
import re
import os

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


---
# Task1: 
Extract mom-ids and baby-ids that have both prenatal visit and postnatal visit, non-missing value of mom's gestational age, baby's birth weight, and baby's date of birth

## Using Dropbox Data

#### Remove babies with any missing value of gestational age, baby's birth or baby's DOB

In [10]:
baby = pd.read_excel(io="../data/Baby.xlsx", sheet_name="Baby")

In [11]:
baby.shape

(16684, 10)

In [12]:
baby.dropna(subset = ["DOB", "Birth Weight (grams)", "Gestational_Age"], inplace=True)

In [13]:
baby.shape

(15829, 10)

#### Extract baby-ids

In [14]:
included_baby_ids = baby["Baby-Id"].unique()

In [24]:
included_baby_ids

array(['Baby-0616', 'Baby-0002', 'Baby-0003', ..., 'Baby-16675',
       'Baby-16676', 'Baby-16677'], dtype=object)

In [23]:
included_baby_ids.shape

(15829,)

#### Extract mom-ids

In [15]:
mom_baby_link = pd.read_excel(io="../data/Baby.xlsx", sheet_name="Baby-Mom Link")

In [16]:
mom_baby_link.head()

,Baby-Id,Mom-Id
0,Baby-0001,Mom-4388
1,Baby-0002,Mom-1829
2,Baby-0003,Mom-8494
3,Baby-0004,Mom-0320
4,Baby-0005,Mom-6297


In [19]:
included_mom_ids = mom_baby_link[mom_baby_link["Baby-Id"].isin(included_baby_ids)]["Mom-Id"].unique()

In [25]:
included_mom_ids

array(['Mom-4388', 'Mom-1829', 'Mom-8494', ..., 'Mom-3097', 'Mom-11509',
       'Mom-11014'], dtype=object)

In [26]:
included_mom_ids.shape

(13251,)

## Save Extracted IDs to Pickle

In [30]:
np.save("selected_mom_ids.npy", included_mom_ids)
np.save("selected_baby_ids.npy", included_baby_ids)

---
# Task2: 
look at mom-baby EHR to find out how many women deliver between May-August on average. Then subset to include gestational ages > 27 weeks to exclude very preterm babies.

In [2]:
baby = pd.read_excel(io="../data/Baby.xlsx", sheet_name="Baby")

In [3]:
link = pd.read_excel(io="../data/Baby.xlsx", sheet_name="Mom-Baby")

In [4]:
baby.head()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS
0,Baby-0616,WHITE,NOT HISPANIC,1902-01-01,3627.0,"C-Section, Low Transverse",2011-12-15,INTRAMURAL BIRTH,39 4/7,NaN
1,Baby-0002,WHITE,NOT HISPANIC,2011-06-01,3417.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,38 3/7,NaN
2,Baby-0003,MULTIRACIAL,HISPANIC,2011-06-01,3383.0,"C-Section, Low Transverse",2011-06-01,INTRAMURAL BIRTH,40 4/7,NaN
3,Baby-0005,WHITE,NOT HISPANIC,2011-06-01,3882.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,NaN,NaN
4,Baby-0006,WHITE,NOT HISPANIC,2011-06-01,3710.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,39 6/7,NaN


In [5]:
link.head()

,Baby-Id,Baby Gender,Mom-Id,Mom Age,Payer
0,Baby-0001,MALE,Mom-4388,25,Blue Cross
1,Baby-0002,FEMALE,Mom-1829,21,Medicaid
2,Baby-0003,MALE,Mom-8494,19,Medicaid
3,Baby-0004,FEMALE,Mom-0320,30,Medicaid
4,Baby-0005,FEMALE,Mom-6297,26,Medicaid


In [11]:
len(set(baby["Baby-Id"]))

16684

In [12]:
len(set(link["Baby-Id"]))

16649

In [13]:
set(baby["Baby-Id"]) - set(link["Baby-Id"])

{'Baby-0197',
 'Baby-0257',
 'Baby-10041',
 'Baby-10118',
 'Baby-10892',
 'Baby-11289',
 'Baby-11344',
 'Baby-11345',
 'Baby-11539',
 'Baby-11659',
 'Baby-11820',
 'Baby-12160',
 'Baby-12311',
 'Baby-12743',
 'Baby-12859',
 'Baby-1320',
 'Baby-13688',
 'Baby-15539',
 'Baby-15789',
 'Baby-1868',
 'Baby-2346',
 'Baby-2381',
 'Baby-2545',
 'Baby-2804',
 'Baby-2831',
 'Baby-2839',
 'Baby-3190',
 'Baby-3409',
 'Baby-3411',
 'Baby-3646',
 'Baby-3647',
 'Baby-3871',
 'Baby-4060',
 'Baby-4340',
 'Baby-4431',
 'Baby-4595',
 'Baby-4873',
 'Baby-4882',
 'Baby-6023',
 'Baby-6200',
 'Baby-6203',
 'Baby-6219',
 'Baby-6244',
 'Baby-6347',
 'Baby-6438',
 'Baby-6454',
 'Baby-6577',
 'Baby-6688',
 'Baby-6719',
 'Baby-6797',
 'Baby-6803',
 'Baby-7314',
 'Baby-7478',
 'Baby-7481',
 'Baby-7632',
 'Baby-8273',
 'Baby-8296',
 'Baby-8369',
 'Baby-9057',
 'Baby-9328',
 'Baby-9424',
 'Baby-9837'}

In [8]:
baby.tail()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS
16679,Baby-16673,BLACK,NOT HISPANIC,2017-04-29,2750.0,"Vaginal, Spontaneous Delivery",2017-04-29,INTRAMURAL BIRTH,38 6/7,NaN
16680,Baby-16674,WHITE,NOT HISPANIC,2017-04-29,3302.0,"Vaginal, Spontaneous Delivery",2017-04-29,INTRAMURAL BIRTH,38 6/7,NaN
16681,Baby-16675,BLACK,NOT HISPANIC,2017-04-30,2981.0,"Vaginal, Spontaneous Delivery",2017-04-30,INTRAMURAL BIRTH,39 2/7,NaN
16682,Baby-16676,BLACK,NOT HISPANIC,2017-04-30,3165.0,"Vaginal, Spontaneous Delivery",2017-04-30,INTRAMURAL BIRTH,41,NaN
16683,Baby-16677,WHITE,NOT HISPANIC,2017-04-30,4310.0,"Vaginal, Spontaneous Delivery",2017-04-30,INTRAMURAL BIRTH,39 6/7,NaN


In [9]:
link.tail()

,Baby-Id,Baby Gender,Mom-Id,Mom Age,Payer
16644,Baby-9995,FEMALE,Mom-8809,28,Blue Cross
16645,Baby-9996,FEMALE,Mom-3097,31,Blue Cross
16646,Baby-9997,FEMALE,Mom-11509,28,Blue Cross
16647,Baby-9998,FEMALE,Mom-11014,36,Managed Care
16648,Baby-9999,MALE,Mom-5254,29,Blue Cross


In [14]:
# merge two dataframe
data = baby.merge(link, on="Baby-Id")

In [15]:
len(data)

16622

In [16]:
data.head()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS,Baby Gender,Mom-Id,Mom Age,Payer
0,Baby-0616,WHITE,NOT HISPANIC,1902-01-01,3627.0,"C-Section, Low Transverse",2011-12-15,INTRAMURAL BIRTH,39 4/7,NaN,MALE,Mom-2592,29,Medicaid
1,Baby-0002,WHITE,NOT HISPANIC,2011-06-01,3417.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,38 3/7,NaN,FEMALE,Mom-1829,21,Medicaid
2,Baby-0003,MULTIRACIAL,HISPANIC,2011-06-01,3383.0,"C-Section, Low Transverse",2011-06-01,INTRAMURAL BIRTH,40 4/7,NaN,MALE,Mom-8494,19,Medicaid
3,Baby-0005,WHITE,NOT HISPANIC,2011-06-01,3882.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,NaN,NaN,FEMALE,Mom-6297,26,Medicaid
4,Baby-0006,WHITE,NOT HISPANIC,2011-06-01,3710.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,39 6/7,NaN,MALE,Mom-9003,25,Managed Care


In [17]:
data["Admit Date"].iloc[0]

Timestamp('2011-12-15 00:00:00')

In [20]:
# remove very preterm babies
data_nonpreterm = data[data.Gestational_Age>"27"]

In [23]:
# Check if the smallest value of gestational age is valid
data_nonpreterm.sort_values(by="Gestational_Age").head()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS,Baby Gender,Mom-Id,Mom Age,Payer
15680,Baby-15735,WHITE,NOT HISPANIC,2016-12-21,930.0,"C-Section, Low Transverse",2016-12-21,INTRAMURAL BIRTH,27 1/7,67.0,FEMALE,Mom-13782,29,Managed Care
13442,Baby-13495,OTHER,HISPANIC,2016-03-07,826.0,NOT INCLUDED IN ORIGINAL SOURCE,2016-03-07,INTRAMURAL BIRTH,27 1/7,72.0,FEMALE,Mom-10020,28,Blue Cross
15125,Baby-15179,BLACK,NOT HISPANIC,2016-10-19,760.0,"C-Section, Low Transverse",2016-10-19,INTRAMURAL BIRTH,27 1/7,32.0,FEMALE,Mom-4995,20,Medicaid HMO
10606,Baby-10651,WHITE,NOT HISPANIC,2015-02-27,824.0,"C-Section, Classical",2015-02-27,INTRAMURAL BIRTH,27 1/7,39.0,FEMALE,Mom-8212,38,Blue Cross
9422,Baby-9461,MULTIRACIAL,NOT HISPANIC,2014-10-01,950.0,"Vaginal, Spontaneous Delivery",2014-10-01,INTRAMURAL BIRTH,27 1/7,81.0,FEMALE,Mom-3167,20,Medicaid HMO


In [24]:
# Check if the biggest value of gestational age is valid
data_nonpreterm.sort_values(by="Gestational_Age").tail()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS,Baby Gender,Mom-Id,Mom Age,Payer
10798,Baby-10841,WHITE,NOT HISPANIC,2015-03-24,3405.0,"Vaginal, Spontaneous Delivery",2015-03-24,INTRAMURAL BIRTH,42 5/7,0.0,FEMALE,Mom-12507,26,Medicaid HMO
7674,Baby-7707,OTHER,HISPANIC,2014-02-04,4530.0,"C-Section, Low Transverse",2014-02-04,INTRAMURAL BIRTH,42 6/7,NaN,MALE,Mom-3221,20,Medicaid
8886,Baby-8922,WHITE,NOT HISPANIC,2014-07-18,3921.0,"Vaginal, Spontaneous Delivery",2014-07-18,INTRAMURAL BIRTH,42 6/7,NaN,MALE,Mom-3485,37,Medicaid HMO
5612,Baby-5634,OTHER,NOT HISPANIC,2013-05-18,3348.0,"Vaginal, Spontaneous Delivery",2013-05-18,INTRAMURAL BIRTH,43,NaN,MALE,Mom-10504,33,SELF PAY
8476,Baby-8512,WHITE,NOT HISPANIC,2014-05-22,2380.0,Extramural Delivery,2014-05-22,EXTRAMURAL BIRTH,84 1/7,NaN,FEMALE,Mom-1974,23,Medicaid HMO


In [29]:
# Check the earliest date of births
data_nonpreterm.sort_values(by="DOB").head()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS,Baby Gender,Mom-Id,Mom Age,Payer
0,Baby-0616,WHITE,NOT HISPANIC,1902-01-01,3627.0,"C-Section, Low Transverse",2011-12-15,INTRAMURAL BIRTH,39 4/7,NaN,MALE,Mom-2592,29,Medicaid
1,Baby-0002,WHITE,NOT HISPANIC,2011-06-01,3417.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,38 3/7,NaN,FEMALE,Mom-1829,21,Medicaid
2,Baby-0003,MULTIRACIAL,HISPANIC,2011-06-01,3383.0,"C-Section, Low Transverse",2011-06-01,INTRAMURAL BIRTH,40 4/7,NaN,MALE,Mom-8494,19,Medicaid
4,Baby-0006,WHITE,NOT HISPANIC,2011-06-01,3710.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,39 6/7,NaN,MALE,Mom-9003,25,Managed Care
5,Baby-0007,HISPANIC,HISPANIC,2011-06-01,3302.0,"Vaginal, Spontaneous Delivery",2011-06-01,INTRAMURAL BIRTH,38 5/7,NaN,FEMALE,Mom-2549,17,Medicaid


In [30]:
# Check the latest date of births
data_nonpreterm.sort_values(by="DOB").tail()

,Baby-Id,Race,Ethnicity,DOB,Birth Weight (grams),Delivery_Mode,Admit Date,Admit Source,Gestational_Age,NICU LOS,Baby Gender,Mom-Id,Mom Age,Payer
16616,Baby-16672,WHITE,NOT HISPANIC,2017-04-29,4286.0,"C-Section, Low Transverse",2017-04-29,INTRAMURAL BIRTH,38 4/7,NaN,MALE,Mom-13390,23,Blue Cross
16618,Baby-16674,WHITE,NOT HISPANIC,2017-04-29,3302.0,"Vaginal, Spontaneous Delivery",2017-04-29,INTRAMURAL BIRTH,38 6/7,NaN,MALE,Mom-3340,22,Managed Care
16620,Baby-16676,BLACK,NOT HISPANIC,2017-04-30,3165.0,"Vaginal, Spontaneous Delivery",2017-04-30,INTRAMURAL BIRTH,41,NaN,MALE,Mom-13670,25,Medicaid HMO
16619,Baby-16675,BLACK,NOT HISPANIC,2017-04-30,2981.0,"Vaginal, Spontaneous Delivery",2017-04-30,INTRAMURAL BIRTH,39 2/7,NaN,FEMALE,Mom-13736,39,Medicaid HMO
16621,Baby-16677,WHITE,NOT HISPANIC,2017-04-30,4310.0,"Vaginal, Spontaneous Delivery",2017-04-30,INTRAMURAL BIRTH,39 6/7,NaN,MALE,Mom-12687,40,Medicaid


Use DOB of 2012-2016 to calculate the average value since 2011 does not have May while 2017 does not have May-August

In [33]:
data_2012_summer = data_nonpreterm[(data_nonpreterm['DOB'] > '2012-05-01 00:00:00') & (data_nonpreterm['DOB'] < '2012-08-31 23:59:59')]
data_2013_summer = data_nonpreterm[(data_nonpreterm['DOB'] > '2013-05-01 00:00:00') & (data_nonpreterm['DOB'] < '2013-08-31 23:59:59')]
data_2014_summer = data_nonpreterm[(data_nonpreterm['DOB'] > '2014-05-01 00:00:00') & (data_nonpreterm['DOB'] < '2014-08-31 23:59:59')]
data_2015_summer = data_nonpreterm[(data_nonpreterm['DOB'] > '2015-05-01 00:00:00') & (data_nonpreterm['DOB'] < '2015-08-31 23:59:59')]
data_2016_summer = data_nonpreterm[(data_nonpreterm['DOB'] > '2016-05-01 00:00:00') & (data_nonpreterm['DOB'] < '2016-08-31 23:59:59')]

In [40]:
# number of babies born during 2012-2016 summer
len(list(data_2012_summer["Mom-Id"]) + \
              list(data_2013_summer["Mom-Id"]) + \
              list(data_2014_summer["Mom-Id"]) + \
              list(data_2015_summer["Mom-Id"]) + \
              list(data_2016_summer["Mom-Id"]))

4233

In [62]:
# number of moms deliverying during 2012-2016 summer
mom_ids = set(list(data_2012_summer["Mom-Id"]) + \
              list(data_2013_summer["Mom-Id"]) + \
              list(data_2014_summer["Mom-Id"]) + \
              list(data_2015_summer["Mom-Id"]) + \
              list(data_2016_summer["Mom-Id"]))
# remove nan from the set
mom_ids = {x for x in mom_ids if x==x}
len(mom_ids)

3963

In [63]:
# average number of moms deliverying during 2012-2016 summer
len(mom_ids)/5

792.6

In [64]:
# average number of moms deliverying during 2012-2016 May
len(data[data.DOB.dt.month==5]["Mom-Id"].ffill().unique())/5

206.0

In [65]:
# average number of moms deliverying during 2011-2016 June
len(data[data.DOB.dt.month==6]["Mom-Id"].ffill().unique())/5

253.8

In [66]:
# average number of moms deliverying during 2011-2016 July
len(data[data.DOB.dt.month==7]["Mom-Id"].ffill().unique())/5

273.4

In [68]:
# average number of moms deliverying during 2011-2016 August
len(data[data.DOB.dt.month==8]["Mom-Id"].ffill().unique())/5

284.2